In [2]:
import os
import pycardano as pc
import random
import time
from marketplace import * 
import pathlib
from vesting import *

Need to build the vesting contract and transaction to deploy it.

In [3]:
NETWORK = pc.Network.TESTNET

# Use ogmios backend for chain context
context = pc.OgmiosChainContext(ws_url="wss://ogmios-preprod-api-testing-2fdd06.us1.demeter.run", network=NETWORK)

In [5]:
"""Preparation"""
# Define the root directory where images and keys will be stored.
PROJECT_ROOT = "marketplace"
root = pathlib.Path(PROJECT_ROOT)

# Create the directory if it doesn't exist
root.mkdir(parents=True, exist_ok=True)

"""Generate keys"""
key_dir = root / "keys"
key_dir.mkdir(exist_ok=True)

In [6]:
# Load payment keys or create them if they don't exist
def load_or_create_key_pair(base_dir, base_name):
    skey_path = base_dir / f"{base_name}.skey"
    vkey_path = base_dir / f"{base_name}.vkey"
    stake_skey_path = base_dir / f"{base_name}_stake.skey"
    stake_vkey_path = base_dir / f"{base_name}_stake.vkey"

    if skey_path.exists():
        skey = pc.PaymentSigningKey.load(str(skey_path))
        vkey = pc.PaymentVerificationKey.from_signing_key(skey)
        stake_skey = pc.StakeSigningKey.load(str(stake_skey_path))
        stake_vkey = pc.StakeVerificationKey.from_signing_key(stake_skey)
    else:
        key_pair = pc.PaymentKeyPair.generate()
        key_pair.signing_key.save(str(skey_path))
        key_pair.verification_key.save(str(vkey_path))
        stake_key_pair = pc.StakeKeyPair.generate()
        stake_key_pair.signing_key.save(str(stake_skey_path))
        stake_key_pair.verification_key.save(str(stake_vkey_path))
        stake_skey = stake_key_pair.signing_key
        stake_vkey = stake_key_pair.verification_key
        skey = key_pair.signing_key
        vkey = key_pair.verification_key
    
    return skey, vkey, stake_skey, stake_vkey

In [7]:
owner_skey, owner_vkey, owner_stake_skey, owner_stake_vkey = load_or_create_key_pair(key_dir, "owner")
beneficiary_skey, beneficiary_vkey, beneficiary_stake_skey, beneficiary_stake_vkey = load_or_create_key_pair(key_dir, "bidder")

In [8]:
owner_address = pc.Address(payment_part=owner_vkey.hash(), staking_part=owner_stake_vkey.hash(), network=NETWORK)
beneficiary_address = pc.Address(payment_part=beneficiary_vkey.hash(), staking_part=beneficiary_stake_vkey.hash(), network=NETWORK)
print(f"Owner address: {owner_address}")
print(f"beneficiary address: {beneficiary_address}")

Owner address: addr_test1qq5g0m0h7kzts8mfuckeuyr2qxk9fgfskz47kx0fz7l45m35sz43s8xuc9qt8kdxlchhy90d4yp0sa43vdrxuu9jewps34ue95
beneficiary address: addr_test1qzjr9whvuyldxf9ft6998jjwnq72syr2yeqlmygxfhl2d2c8lxhg9ar5qfnk2w5j0gj6fvygkdcdgd9e7gftjgt4dvhqv0ej3s


In [9]:
vkey_hash: pc.VerificationKeyHash = beneficiary_address.payment_part

In [16]:
time.time()

1680226754.0247638

In [24]:
# Create the vesting datum
params = VestingParams(
    beneficiary=bytes(vkey_hash),
    deadline=int(time.time() + 5) * 1000,  # must be in milliseconds
)

In [25]:
script_path = pathlib.Path().cwd()
script_path = script_path.joinpath("build","vesting", "testnet.addr")

In [26]:
# Load script info
with open(script_path) as f:
        script_address = pc.Address.from_primitive(f.read())

In [27]:
datum = params

In [28]:
# Build the transaction
builder = pc.TransactionBuilder(context)
builder.add_input_address(owner_address)
builder.add_output(
        pc.TransactionOutput(address=script_address, amount=3000000, datum=datum)
    )



TransactionBuilder(context=<pycardano.backend.ogmios.OgmiosChainContext object at 0x7f8d20e82d90>, utxo_selectors=[<pycardano.coinselection.RandomImproveMultiAsset object at 0x7f8cf09db730>, <pycardano.coinselection.LargestFirstSelector object at 0x7f8d20e821c0>], execution_memory_buffer=0.2, execution_step_buffer=0.2, ttl=None, validity_start=None, auxiliary_data=None, native_scripts=None, mint=None, required_signers=None, collaterals=[], certificates=None, withdrawals=None, reference_inputs=set(), _inputs=[], _excluded_inputs=[], _input_addresses=[addr_test1qq5g0m0h7kzts8mfuckeuyr2qxk9fgfskz47kx0fz7l45m35sz43s8xuc9qt8kdxlchhy90d4yp0sa43vdrxuu9jewps34ue95], _outputs=[{
  'address': addr_test1wzx39q3s0k964gf3crsjtzfvej5tpgzl3pnnkg2hw2t96fc0wdyw8,
  'amount': {'coin': 3000000, 'multi_asset': {}},
  'datum': VestingParams(beneficiary=b'\xa42\xba\xec\xe1>\xd3$\xa9^\x8aS\xcaN\x98<\xa8\x10j&A\xfd\x91\x06M\xfe\xa6\xab', deadline=1680226821000),
  'datum_hash': None,
  'post_alonzo': False,
 

In [33]:
# Sign the transaction
signed_tx = builder.build_and_sign(
    signing_keys=[owner_skey],
    change_address=owner_address,
)

print("############### Transaction created ###############")
print(signed_tx)
print(signed_tx.to_cbor())

############### Transaction created ###############
{
  'auxiliary_data': None,
  'transaction_body': {
    'auxiliary_data_hash': None,
    'certificates': None,
    'collateral': None,
    'collateral_return': None,
    'fee': 169109,
    'inputs': [
      {
        'index': 1,
        'transaction_id': TransactionId(hex='09a87ffaab78b8ba90c240650af4662cbdb8543b7731a73b54b487314e929479'),
      },
    ],
    'mint': None,
    'network_id': None,
    'outputs': [
      {
        'address': addr_test1wzx39q3s0k964gf3crsjtzfvej5tpgzl3pnnkg2hw2t96fc0wdyw8,
        'amount': {'coin': 3000000, 'multi_asset': {}},
        'datum': VestingParams(beneficiary=b'\xa42\xba\xec\xe1>\xd3$\xa9^\x8aS\xcaN\x98<\xa8\x10j&A\xfd\x91\x06M\xfe\xa6\xab', deadline=1680226821000),
        'datum_hash': None,
        'post_alonzo': False,
        'script': None,
      },
      {
        'address': addr_test1qq5g0m0h7kzts8mfuckeuyr2qxk9fgfskz47kx0fz7l45m35sz43s8xuc9qt8kdxlchhy90d4yp0sa43vdrxuu9jewps34ue95,
   

In [34]:
# Submit the transaction
# context.submit_tx(signed_tx.to_cbor())

print(f"transaction id: {signed_tx.id}")
print(f"Cardanoscan: https://preprod.cardanoscan.io/transaction/{signed_tx.id}")

transaction id: 56e8aa4dcdbb5b7c515b875621de26f0ed7befb34552c9432071d863ce48f23b
Cardanoscan: https://preprod.cardanoscan.io/transaction/56e8aa4dcdbb5b7c515b875621de26f0ed7befb34552c9432071d863ce48f23b
